In [1]:
!uv pip install pandas

Audited 1 package in 5ms


In [2]:
import pandas as pd

In [18]:
df_truth = pd.read_csv("data/echantillon_eval_hs_2024_contingent_annote.csv")
df_truth = df_truth.rename(columns={"numdossier_new":"num_dossier"})
df_truth = df_truth.rename(columns={"contingent":"contingent_truth"})
df_truth["contingent_truth"] = df_truth["contingent_truth"].fillna("")

df_merged = df_truth[['num_dossier']].copy()

# Ajouter les colonnes des résultats RAG pour chaque k
for k in range(1, 11):
    df_k = pd.read_csv(f"contingents_annuels_article_k={k}.csv")
    df_k["contingent_annuel"] = df_k["contingent_annuel"].fillna("")
    df_merged = df_merged.merge(df_k.rename(columns={"contingent_annuel": f"contingent_k{k}"}), on="num_dossier", how="left")


# Ajouter la vérité terrain
df_merged = df_merged.merge(df_truth, on="num_dossier", how="left")


In [19]:
df_merged

,num_dossier,contingent_k1,contingent_k2,contingent_k3,contingent_k4,contingent_k5,contingent_k6,contingent_k7,contingent_k8,contingent_k9,contingent_k10,contingent_truth
0,T00724060165,200.0,,,35.0,,,50.0,200.0,,25.0,
1,T03324062903,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0
2,T07424060533,,,,60.0,,,,,,,
3,T03224060321,,,,,,,,,,,
4,T06724061948,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0,360.0
...,...,...,...,...,...,...,...,...,...,...,...,...
95,T09224067019,,,,,,,,,,,
96,T09224068199,,,200.0,,200.0,,,,,,
97,T04524060557,220.0,220.0,220.0,220.0,220.0,220.0,220.0,220.0,220.0,220.0,220.0
98,T59L24063310,,,,,,,,,,,


In [20]:
# Créer un DataFrame avec une colonne booléenne pour chaque k indiquant si c'est incorrect
for k in range(1, 11):
    df_merged[f"err_k{k}"] = df_merged[f"contingent_k{k}"] != df_merged["contingent_truth"]


In [21]:
# Une ligne est "en erreur" s’il y a AU MOINS UNE erreur sur les 10 k
df_merged["erreur_globale"] = df_merged[[f"err_k{k}" for k in range(1, 11)]].any(axis=1)

# Garder uniquement ces cas
df_erreurs = df_merged[df_merged["erreur_globale"]]


In [22]:
df_erreurs

,num_dossier,contingent_k1,contingent_k2,contingent_k3,contingent_k4,contingent_k5,contingent_k6,contingent_k7,contingent_k8,contingent_k9,...,err_k2,err_k3,err_k4,err_k5,err_k6,err_k7,err_k8,err_k9,err_k10,erreur_globale
0,T00724060165,200.0,,,35.0,,,50.0,200.0,,...,False,False,True,False,False,True,True,False,True,True
2,T07424060533,,,,60.0,,,,,,...,False,False,True,False,False,False,False,False,False,True
12,T07424061503,300.0,300.0,300.0,300.0,300.0,300.0,300.0,,,...,False,False,False,False,False,False,True,True,True,True
19,T00624060638,250.0,250.0,250.0,250.0,250.0,250.0,250.0,250.0,,...,False,False,False,False,False,False,False,True,True,True
22,T04524060711,461.0,461.0,461.0,200.0,200.0,461.0,200.0,200.0,461.0,...,False,False,True,True,False,True,True,False,False,True
23,T02524061095,160.0,72.0,72.0,72.0,72.0,72.0,72.0,,,...,True,True,True,True,True,True,False,False,False,True
32,T07724009353,,,,,,,,,,...,False,False,False,False,False,False,False,False,True,True
37,T03124062604,360.0,360.0,360.0,,,,,,,...,False,False,True,True,True,True,True,True,True,True
38,T04424061694,220.0,220.0,220.0,220.0,220.0,220.0,220.0,,,...,False,False,False,False,False,False,True,True,True,True
53,T03324061515,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,400.0,...,True,True,True,True,True,True,True,True,True,True


In [24]:
df_erreurs.to_csv('resulats.csv', index=False)  
